In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder
  .appName("SummarizerExample")
  .getOrCreate()

import spark.implicits._

spark = org.apache.spark.sql.SparkSession@1ad41540


org.apache.spark.sql.SparkSession@1ad41540

In [2]:
val df = sc.parallelize(Seq(
  ("tom", 0.3, 1.0), ("tom", 0.5, 0.0),
  ("tim", 0.6, 0.5), ("tim", 5.6, 0.2))
).toDF("col1", "col2", "col3")

df.groupBy($"col1").min().show
df.groupBy($"col1").sum().show

+----+---------+---------+
|col1|min(col2)|min(col3)|
+----+---------+---------+
| tim|      0.6|      0.2|
| tom|      0.3|      0.0|
+----+---------+---------+

+----+-----------------+---------+
|col1|        sum(col2)|sum(col3)|
+----+-----------------+---------+
| tim|6.199999999999999|      0.7|
| tom|              0.8|      1.0|
+----+-----------------+---------+



df = [col1: string, col2: double ... 1 more field]


[col1: string, col2: double ... 1 more field]

In [5]:
df.groupBy("col1").sum("col2", "col3").show()

+----+-----------------+---------+
|col1|        sum(col2)|sum(col3)|
+----+-----------------+---------+
| tim|6.199999999999999|      0.7|
| tom|              0.8|      1.0|
+----+-----------------+---------+



In [6]:
val exprs = df.columns.map((_ -> "mean")).toMap
df.groupBy($"col1").agg(exprs).show()

+----+---------+------------------+---------+
|col1|avg(col1)|         avg(col2)|avg(col3)|
+----+---------+------------------+---------+
| tim|     null|3.0999999999999996|     0.35|
| tom|     null|               0.4|      0.5|
+----+---------+------------------+---------+



exprs = Map(col1 -> mean, col2 -> mean, col3 -> mean)


Map(col1 -> mean, col2 -> mean, col3 -> mean)

In [8]:
import org.apache.spark.sql.functions.sum

val exprs = df.columns.map(sum(_))
df.groupBy($"col1").agg(exprs.head, exprs.tail: _*).show()

+----+---------+-----------------+---------+
|col1|sum(col1)|        sum(col2)|sum(col3)|
+----+---------+-----------------+---------+
| tim|     null|6.199999999999999|      0.7|
| tom|     null|              0.8|      1.0|
+----+---------+-----------------+---------+



exprs = Array(sum(col1), sum(col2), sum(col3))


Array(sum(col1), sum(col2), sum(col3))

In [14]:
import org.apache.spark.sql.functions._
df.groupBy($"col1").agg(
    sum("col2").alias("col2_sum"), 
    sum("col3").alias("col3_sum"), 
    avg("col3").alias("col3_avg"))
    .show()

+----+-----------------+--------+--------+
|col1|         col2_sum|col3_sum|col3_avg|
+----+-----------------+--------+--------+
| tim|6.199999999999999|     0.7|    0.35|
| tom|              0.8|     1.0|     0.5|
+----+-----------------+--------+--------+



See the example below - Using Maps

```scala
val Claim1 = StructType(Seq(StructField("pid", StringType, true),StructField("diag1", StringType, true),StructField("diag2", StringType, true), StructField("allowed", IntegerType, true), StructField("allowed1", IntegerType, true)))
val claimsData1 = Seq(("PID1", "diag1", "diag2", 100, 200), ("PID1", "diag2", "diag3", 300, 600), ("PID1", "diag1", "diag5", 340, 680), ("PID2", "diag3", "diag4", 245, 490), ("PID2", "diag2", "diag1", 124, 248))

val claimRDD1 = sc.parallelize(claimsData1)
val claimRDDRow1 = claimRDD1.map(p => Row(p._1, p._2, p._3, p._4, p._5))
val claimRDD2DF1 = sqlContext.createDataFrame(claimRDDRow1, Claim1)

val l = List("allowed", "allowed1")
val exprs = l.map((_ -> "sum")).toMap
claimRDD2DF1.groupBy("pid").agg(exprs) show false
val exprs = Map("allowed" -> "sum", "allowed1" -> "avg")

claimRDD2DF1.groupBy("pid").agg(exprs) show false
```